1- Préparation des données

In [123]:
import pandas as pd 
import numpy as np
 
#chargement de la base de données
df= pd.read_csv("C:/Users/wague/Downloads/projetia/Project_ISEN_Big_Data-IA-Web/clean_IA.csv", parse_dates=['BaseDateTime'])
#parse_dates permet de convertir la colonne BaseDateTime en format datetime

#Tri des données par date et par MMSI
df = df.sort_values(by=['MMSI','BaseDateTime'])

#Variables importantes: LAT, LON, SOG, COG, Heading, Length, Draft
#Variables dépendantes à prédire: LAT et LON
targets = ['LAT', 'LON']
#Variables explicatives: SOG, COG, Heading, potentiellement enrichi par: VesselType, Length, Draft
features = ['LAT', 'LON','SOG', 'COG', 'Heading','Length', 'Draft']
print(df.head(30))

              id       MMSI         BaseDateTime       LAT       LON  SOG  \
86832    9067786  205776000  2023-05-25 00:07:27  25.95847 -97.37876  0.0   
87074    9094320  205776000  2023-05-25 00:13:27  25.95846 -97.37880  0.0   
87231    9113012  205776000  2023-05-25 00:16:27  25.95847 -97.37880  0.0   
92057    9625845  205776000  2023-05-25 00:31:27  25.95848 -97.37879  0.0   
88594    9261545  205776000  2023-05-25 00:34:27  25.95846 -97.37882  0.0   
88237    9223310  205776000  2023-05-25 00:37:27  25.95848 -97.37882  0.0   
88591    9261435  205776000  2023-05-25 00:40:27  25.95846 -97.37883  0.0   
89174    9321671  205776000  2023-05-25 00:43:27  25.95849 -97.37879  0.0   
89348    9340357  205776000  2023-05-25 00:46:27  25.95846 -97.37878  0.1   
89095    9314767  205776000  2023-05-25 00:55:27  25.95842 -97.37880  0.0   
89529    9359156  205776000  2023-05-25 01:01:27  25.95841 -97.37882  0.1   
89476    9353676  205776000  2023-05-25 01:04:27  25.95846 -97.37879  0.1   

In [109]:
#Afin de prédire la position du navire toutes les 5 minutes
period=5
x,y_5,y_10,y_15 = [], [], [], []
for mmsi, group in df.groupby('MMSI'):
    #On récupère les données du navire
    group = group.sort_values(by='BaseDateTime')
    
    for i in range(len(group) - period -3):
    #On récupère les données de la période de 5 minutes
        x_period = group.iloc[i:i + 2][features].values 
        y_5_period = group.iloc[i+1][targets].values
        y_10_period = group.iloc[i+2][targets].values
        y_15_period = group.iloc[i+3][targets].values
        #On ajoute les valeurs à la liste globale
        x.append(x_period)
        y_5.append(y_5_period)
        y_10.append(y_10_period)
        y_15.append(y_15_period)

x = np.array(x)
y_5 = np.array(y_5)
y_10 = np.array(y_10)
y_15 = np.array(y_15)


2- Apprentissage supervisé du modèle

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor

#On reshape les données pour qu'elles soient compatibles avec les modèles
x = x.reshape(x.shape[0], -1)  # C'est sous le forme (n_ecahantillons, n_features)

In [ ]:
#Séparation en données d'entrainement (80%) et données de test (20%)

x_train,x_test,y_5_train,y_5_test=train_test_split(x, y_5, test_size=0.2, random_state=42)
x_train,x_test,y_10_train,y_10_test=train_test_split(x, y_10, test_size=0.2, random_state=42)
x_train,x_test,y_15_train,y_15_test=train_test_split(x, y_15, test_size=0.2, random_state=42)

In [82]:
y_all = np.hstack([y_5_train, y_10_train, y_15_train]) 
model1=RandomForestRegressor().fit(x_train, y_all) 

In [ ]:
#Entrainement de différents modèles avec y_5 
model1=RandomForestRegressor().fit(x_train, y_5_train) 
model2=MultiOutputRegressor(RandomForestRegressor()).fit(x_train, y_5_train)
model3=DecisionTreeRegressor().fit(x_train, y_5_train) 
model4=LinearRegression().fit(x_train, y_5_train)

#Prédiction des données de test avec les modèles entrainés
y_5_pred1 = model1.predict(x_test)
y_5_pred2 = model2.predict(x_test) 
y_5_pred3 = model3.predict(x_test)
y_5_pred4 = model4.predict(x_test)
#Peut etre afficher aussi le temps d'apprentissage de chaque modèle

In [100]:
#Evaluation des modèles sur les données d'apprentissage
mse_5_1 = mean_squared_error(y_5_test, y_5_pred1)
mse_5_2 = mean_squared_error(y_5_test, y_5_pred2)
mse_5_3 = mean_squared_error(y_5_test, y_5_pred3)
mse_5_4 = mean_squared_error(y_5_test, y_5_pred4)


rmse_5_1 = np.sqrt(mse_5_1) 
rmse_5_2 = np.sqrt(mse_5_2)
rmse_5_3 = np.sqrt(mse_5_3)
rmse_5_4 = np.sqrt(mse_5_4)
# Affichage des résultats
print(f"Root Mean Squared Error pour Random Forest: {rmse_5_1}")
print(f"Root Mean Squared Error pour Random Forest Multiple Output: {rmse_5_2}")
print(f"Root Mean Squared Error pour Decision Tree: {rmse_5_3}")
print(f"Root Mean Squared Error pour Linear Regression: {rmse_5_4}")

Root Mean Squared Error pour Random Forest: 0.004790157245282562
Root Mean Squared Error pour Random Forest Multiple Output: 0.012133151963541526
Root Mean Squared Error pour Decision Tree: 0.006472452322404998
Root Mean Squared Error pour Linear Regression: 2.4151419090324252e-14


Random Forest  semble etre le modèle de regression le plus performant car la somme des erreurs résiduelles est nulle. Afin de confirmer cette remarque, on va tester la généralisation

In [108]:
#Evaluation des modèles sur les données de test
#Avec la validation croisée
from sklearn.model_selection import cross_val_score
from math import sqrt
from statistics import mean
#Model 1
scores1 = cross_val_score(model1, x_train, y_5_train, cv=5, scoring='neg_mean_squared_error')
RMSE_1=[sqrt(-x) for x in scores1]  #??On prend la racine carrée de la valeur négative pour obtenir le RMSE
print("Pour le Random Forest:", mean(RMSE_1))
#Model 2
scores2 = cross_val_score(model2, x_train, y_5_train, cv=5, scoring='neg_mean_squared_error')
RMSE_2=[sqrt(-x) for x in scores2]  #??On prend la racine carrée de la valeur négative pour obtenir le RMSE
print("Pour le Multiple Regressor:", mean(RMSE_2))
#Model 3
scores3 = cross_val_score(model3, x_train, y_5_train, cv=5, scoring='neg_mean_squared_error')
RMSE_3=[sqrt(-x) for x in scores2]
print("Pour le Decision Tree:", mean(RMSE_3))
#Model 4
scores4 = cross_val_score(model4, x_train, y_5_train, cv=5, scoring='neg_mean_squared_error')
RMSE_4=[sqrt(-x) for x in scores3] 
print("Pour le Linear Regression:", mean(RMSE_4))


Pour le Random Forest: 0.0074527720629739376
Pour le Multiple Regressor: 0.022481851682332018
Pour le Decision Tree: 0.022481851682332018
Pour le Linear Regression: 0.011901803120324742


Nous allons utiliser Random Forest

In [116]:
#Entrainement de Random Forest avec y_10
model1_10=RandomForestRegressor().fit(x_train, y_10_train) 
y_10_pred1= model1_10.predict(x_test)
#Entrainement de Random Forest avec y_15
model1_15=RandomForestRegressor().fit(x_train, y_15_train) 
y_15_pred1= model1_15.predict(x_test)

Evaluation de la précision

In [117]:
from sklearn.metrics import r2_score

R2_lat = r2_score(y_5_test, y_5_pred1)  # Pour la latitude
R2_lon = r2_score(y_5_test, y_5_pred1)  # Pour la longitude

print(f"R² Latitude  : {r2_lat:.3f}")
print(f"R² Longitude : {r2_lon:.3f}")

R² Latitude  : 1.000
R² Longitude : 1.000


R²=1, ça veut dire que la prédiction est parfaite

3- Enregistrement du modèle

In [118]:
import joblib
#Enregistrement du modèle
joblib.dump(model1, 'model_5.pkl')
joblib.dump(model1_10, 'model_10.pkl')
joblib.dump(model1_15, 'model_15.pkl')

['model_15.pkl']